# HuggingFace `nlp` library

Models come and go (linear models, LSTM, Transformers, ...) but two core elements have consistently been the beating heart of Natural Language Processing: Datasets & Metrics

`nlp` is a lightweight and extensible library to easily share and load dataset and evaluation metrics, already providing access to ~100 datasets.

With several interesting features:

- Get them all: build-in interoperability with PyTorch, Tensorflow 2, Pandas and Numpy
- Transparent and pythonic API
- Smart caching: never process your data several times with an intelligent `tf.data`-like cache
- Strive on large datasets: nlp naturally frees you from RAM memory limits, all datasets are memory-mapped on drive by default.

`nlp` originated from a fork of the awesome Tensorflow-Datasets and the HuggingFace team want to deeply thank the team behind this amazing library and user API. We have tried to keep a layer of compatibility with `tfds` and a conversion can provide conversion from one format to the other.

# Main user API

This notebook is a quick dive in the main user API for `nlp`

In [1]:
import logging
logging.basicConfig(level=logging.INFO)

In [2]:
# Let's import the library
import nlp

INFO:nlp.utils.file_utils:PyTorch version 1.4.0 available.
INFO:nlp.utils.file_utils:TensorFlow version 2.1.0 available.


## Listing the currently available datasets and metrics

In [3]:
# Currently available datasets and metrics
datasets = nlp.list_datasets()
metrics = nlp.list_metrics()

print(f"🤩 Currently {len(datasets)} datasets are available on HuggingFace AWS bucket: \n" 
      + '\n'.join(dataset.id for dataset in datasets) + '\n')
print(f"🤩 Currently {len(metrics)} metrics are available on HuggingFace AWS bucket: \n" 
      + '\n'.join(metric.id for metric in metrics))

🤩 Currently 75 datasets are available on HuggingFace AWS bucket: 
aeslc
ai2_arc
anli
billsum
blimp
blog_authorship_corpus
boolq
break_data
cfq
civil_comments
coarse_discourse
com_qa
commonsense_qa
coqa
cornell_movie_dialog
cos_e
cosmos_qa
crime_and_punish
definite_pronoun_resolution
drop
empathetic_dialogues
eraser_multi_rc
esnli
event2Mind
flores
fquad
gap
glue
hansards
hellaswag
imdb
jeopardy
math_dataset
mlqa
movie_rationales
multi_nli
multi_nli_mismatch
openbookqa
opinosis
para_crawl
qangaroo
qasc
quarel
quartz
quoref
race
scan
scicite
scifact
sciq
scitail
sentiment140
snli
squad
squad_it
squad_v1_pt
squad_v2
super_glue
ted_hrlr
ted_multi
tiny_shakespeare
tydiqa
wiki40b
wiki_qa
wiki_split
wikipedia
wikitext
winogrande
wiqa
x_stance
xcopa
xnli
xquad
xtreme
yelp_polarity

🤩 Currently 10 metrics are available on HuggingFace AWS bucket: 
bleu
coval
gleu
glue
rouge
sacrebleu
seqeval
squad
squad_v2
xnli


In [4]:
# You can read a few attributes of the datasets before loading them (they are python dataclasses)
from dataclasses import asdict

for key, value in asdict(datasets[6]).items():
    print('👉 ' + key + ': ' + str(value))

👉 id: boolq
👉 key: nlp/boolq/boolq.py
👉 lastModified: 2020-05-13T10:40:19.000Z
👉 description: \
BoolQ is a question answering dataset for yes/no questions containing 15942 examples. These questions are naturally 
occurring ---they are generated in unprompted and unconstrained settings. 
Each example is a triplet of (question, passage, answer), with the title of the page as optional additional context. 
The text-pair classification setup is similar to existing natural language inference tasks.
👉 citation: \
@inproceedings{clark2019boolq,
  title =     {BoolQ: Exploring the Surprising Difficulty of Natural Yes/No Questions},
  author =    {Clark, Christopher and Lee, Kenton and Chang, Ming-Wei, and Kwiatkowski, Tom and Collins, Michael, and Toutanova, Kristina},
  booktitle = {NAACL},
  year =      {2019},
}
👉 size: 3444
👉 etag: "bf50ccf7a03c3167addb1032f83f883a"
👉 siblings: [{'key': 'nlp/boolq/boolq.py', 'etag': '"bf50ccf7a03c3167addb1032f83f883a"', 'lastModified': '2020-05-13T10:40:19.

## An example with SQuAD

In [5]:
# Downloading and loading a dataset

dataset = nlp.load_dataset('squad', split='validation[:10%]')

INFO:nlp.load:Checking /Users/thomwolf/.cache/huggingface/datasets/ee43d2be6898ebb9c2afefda4455306911d308bcf924d21c975796832cc7c114.f373b0de1570ca81b50bb03bd371604f7979e35de2cfcf2a3b4521d0b3104d9b.py for additional imports.
INFO:filelock:Lock 5830637136 acquired on /Users/thomwolf/.cache/huggingface/datasets/ee43d2be6898ebb9c2afefda4455306911d308bcf924d21c975796832cc7c114.f373b0de1570ca81b50bb03bd371604f7979e35de2cfcf2a3b4521d0b3104d9b.py.lock
INFO:nlp.load:Found main folder for dataset https://s3.amazonaws.com/datasets.huggingface.co/nlp/squad/squad.py at /Users/thomwolf/Documents/GitHub/datasets/src/nlp/datasets/squad
INFO:nlp.load:Found specific version folder for dataset https://s3.amazonaws.com/datasets.huggingface.co/nlp/squad/squad.py at /Users/thomwolf/Documents/GitHub/datasets/src/nlp/datasets/squad/c0327553d80335e3a3283527f64d9778df7ad04ab28f38148d072782712bb670
INFO:nlp.load:Found script file from https://s3.amazonaws.com/datasets.huggingface.co/nlp/squad/squad.py to /Users/

This call to `nlp.load_dataset()` does the following steps under the hood:

1. Download and import in the library the **SQuAD python processing script** from HuggingFace AWS bucket if it's not already stored in the library. You can find the SQuAD processing script [here](https://github.com/huggingface/nlp/tree/master/datasets/squad/squad.py) for instance.

   Processing scripts are small python scripts which define the info (citation, description) and format of the dataset and contain the URL to the original SQuAD JSON files and the code to load examples from the original SQuAD JSON files.


2. Run the SQuAD python processing script which will:
    - **Download the SQuAD dataset** from the original URL (see the script) if it's not already downloaded and cached.
    - **Process and cache** all SQuAD in a structured Arrow table for each standard splits stored on the drive.

      Arrow table are arbitrarly long tables, typed with types that can be mapped to numpy/pandas/python standard types and can store nested objects. They can be directly access from drive, loaded in RAM or even streamed over the web.
    

3. Return a **dataset build from the splits** asked by the user (default: all), in the above example we create a dataset with the first 10% of the validation split.

In [6]:
# Informations on the dataset (description, citation, size, splits, format...)
# are provided in `dataset.info` (as a python dataclass)
for key, value in asdict(dataset.info).items():
    print('👉 ' + key + ': ' + str(value))

👉 description: Stanford Question Answering Dataset (SQuAD) is a reading comprehension dataset, consisting of questions posed by crowdworkers on a set of Wikipedia articles, where the answer to every question is a segment of text, or span, from the corresponding reading passage, or the question might be unanswerable.

👉 citation: @article{2016arXiv160605250R,
       author = {{Rajpurkar}, Pranav and {Zhang}, Jian and {Lopyrev},
                 Konstantin and {Liang}, Percy},
        title = "{SQuAD: 100,000+ Questions for Machine Comprehension of Text}",
      journal = {arXiv e-prints},
         year = 2016,
          eid = {arXiv:1606.05250},
        pages = {arXiv:1606.05250},
archivePrefix = {arXiv},
       eprint = {1606.05250},
}

👉 homepage: https://rajpurkar.github.io/SQuAD-explorer/
👉 license: 
👉 features: {'id': {'dtype': 'string', 'id': None, '_type': 'Value'}, 'title': {'dtype': 'string', 'id': None, '_type': 'Value'}, 'context': {'dtype': 'string', 'id': None, '_type': 'Va

## Inspecting and using the dataset: elements, slices and columns

The returned `Dataset` object is a memory mapped dataset that behave similarly to a normal map-style dataset. It is backed by an Apache Arrow table which allows many interesting features.

In [7]:
print(dataset)

Dataset(schema: {'id': 'string', 'title': 'string', 'context': 'string', 'question': 'string', 'answers': 'struct<text: list<item: string>, answer_start: list<item: int32>>'}, num_rows: 1057)


You can query it's length and get items or slices like you would do normally with a python mapping.

In [8]:
from pprint import pprint

print(f"👉Dataset len(dataset): {len(dataset)}")
print("\n👉First item 'dataset[0]':")
pprint(dataset[0])
print("\n👉Slice of the first two items 'dataset[:2]':")
pprint(dataset[:2])

👉Dataset len(dataset): 1057

👉First item 'dataset[0]':
{'answers': {'answer_start': [177, 177, 177],
             'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos']},
 'context': 'Super Bowl 50 was an American football game to determine the '
            'champion of the National Football League (NFL) for the 2015 '
            'season. The American Football Conference (AFC) champion Denver '
            'Broncos defeated the National Football Conference (NFC) champion '
            'Carolina Panthers 24–10 to earn their third Super Bowl title. The '
            "game was played on February 7, 2016, at Levi's Stadium in the San "
            'Francisco Bay Area at Santa Clara, California. As this was the '
            '50th Super Bowl, the league emphasized the "golden anniversary" '
            'with various gold-themed initiatives, as well as temporarily '
            'suspending the tradition of naming each Super Bowl game with '
            'Roman numerals (under which 

You can get a full column of the dataset by indexing with its name as a string:

In [9]:
print(dataset['question'][:10])

['Which NFL team represented the AFC at Super Bowl 50?', 'Which NFL team represented the NFC at Super Bowl 50?', 'Where did Super Bowl 50 take place?', 'Which NFL team won Super Bowl 50?', 'What color was used to emphasize the 50th anniversary of the Super Bowl?', 'What was the theme of Super Bowl 50?', 'What day was the game played on?', 'What is the AFC short for?', 'What was the theme of Super Bowl 50?', 'What does AFC stand for?']


The `__getitem__` method will return different format depending on the type of query:

- Items like `dataset[0]` are returned as dict of element.
- Slices like `dataset[10:20]` are returned as dict of lists of elements.
- Columns like `dataset['question']` are returned as a list.

This may seems surprising at first but in our experiments it's actually a lot easier to use for data processing that returning always the same format.

In particular, you can easily iterate along slices and also naturally permute slice, index and columns indexings with identical results:

In [10]:
print(dataset[0]['question'] == dataset['question'][0])
print(dataset[10:20]['context'] == dataset['context'][10:20])

True
True


### Typing

The dataset is backed by an Apache Arrow table which is typed and allows for fast retrieval and acces as well as arbitrary size memory mapping. This means respectively taht the format for the dataset is clearly defined and that you can load datasets of arbitrary size without worrying about RAM memory limitation.

In [11]:
# You can inspect the dataset column names and type 
print(dataset.column_names)
print(dataset.schema)

['id', 'title', 'context', 'question', 'answers']
id: string not null
title: string not null
context: string not null
question: string not null
answers: struct<text: list<item: string>, answer_start: list<item: int32>> not null
  child 0, text: list<item: string>
      child 0, item: string
  child 1, answer_start: list<item: int32>
      child 0, item: int32


### Additional misc properties

In [12]:
# Datasets also have a bunch of properties you can access
print("The number of bytes allocated on the drive is ", dataset.nbytes)
print("For comparison, here is the number of bytes allocated in memory which can be")
print("accessed with `nlp.total_allocated_bytes()`: ", nlp.total_allocated_bytes())
print("The number of rows", dataset.num_rows)
print("The number of columns", dataset.num_columns)
print("The shape (rows, columns)", dataset.shape)

The number of bytes allocated on the drive is  10472672
For comparison, here is the number of bytes allocated in memory which can be
accessed with `nlp.total_allocated_bytes()`:  0
The number of rows 1057
The number of columns 5
The shape (rows, columns) (1057, 5)


### Additional misc methods

In [13]:
# We can list the unique elements in a column. This is done by the backend (so fast!)
print(f"dataset.unique('title'): {dataset.unique('title')}")

# This will drop the column 'id'
dataset.drop('id')  # Remove column 'id'
print(f"After dataset.drop('id'), remaining columns are {dataset.column_names}")

# This will flatten the nested columns in 'answers'
dataset.flatten()
print(f"After dataset.flatten(), column names are {dataset.column_names}")

# We can also "dictionnary encode" a column if many of it's elements are similar
# This will reduce it's size by only storing the distinct elements (e.g. string)
# It only has effect on the internal storage (no difference from a user point of view)
dataset.dictionary_encode_column('title')

dataset.unique('title'): ['Super_Bowl_50', 'Warsaw']
After dataset.drop('id'), remaining columns are ['title', 'context', 'question', 'answers']
After dataset.flatten(), column names are ['title', 'context', 'question', 'answers.text', 'answers.answer_start']


## Cache

`nlp` datasets are backed by Apache Arrow cache files which allows:
- to load arbitrary large datasets by using [memory mapping](https://en.wikipedia.org/wiki/Memory-mapped_file) (as long as the datasets can fit on the drive)
- to use a fast backend to process the dataset efficiently
- to do smart caching by storing and reusing the results of operations performed on the drive

Let's dive a bit in these parts now

You can check the current cache files backing the dataset with the `.cache_file` property

In [14]:
dataset.cache_files

({'filename': '/Users/thomwolf/.cache/huggingface/datasets/squad/plain_text/1.0.0/squad-validation.arrow',
  'skip': 0,
  'take': 1057},)

You can clean up the cache files for in the current dataset directory with the `.cleanup_cache_files()`.

Be careful that no other process is using these cache files when running this command.

In [15]:
dataset.cleanup_cache_files()  # Returns the number of removed cache files

INFO:nlp.arrow_dataset:Listing files in /Users/thomwolf/.cache/huggingface/datasets/squad/plain_text/1.0.0
INFO:nlp.arrow_dataset:Removing /Users/thomwolf/.cache/huggingface/datasets/squad/plain_text/1.0.0/cache-0be0bac8de2ddc42bcc650f62286e93a.arrow
INFO:nlp.arrow_dataset:Removing /Users/thomwolf/.cache/huggingface/datasets/squad/plain_text/1.0.0/cache-b5fef4a553eb27e130e299608150b28a.arrow
INFO:nlp.arrow_dataset:Removing /Users/thomwolf/.cache/huggingface/datasets/squad/plain_text/1.0.0/cache-ce21fee997fe18970e541e2abf4e938f.arrow
INFO:nlp.arrow_dataset:Removing /Users/thomwolf/.cache/huggingface/datasets/squad/plain_text/1.0.0/cache-ca7a1d9079c4c5bcb746ee34fb9587b6.arrow
INFO:nlp.arrow_dataset:Removing /Users/thomwolf/.cache/huggingface/datasets/squad/plain_text/1.0.0/cache-522d9b9a4a2809ce238350c7bff6e151.arrow
INFO:nlp.arrow_dataset:Removing /Users/thomwolf/.cache/huggingface/datasets/squad/plain_text/1.0.0/cache-22f66fb97029056f964fa6c0093271ff.arrow
INFO:nlp.arrow_dataset:Removi

8

## Modifying the dataset with `dataset.map`

There is a powerful method `.map()` that you can use to apply a function to each examples, independantly or in batch.

In [16]:
# `.map()` takes a callable accepting a dict as argument
# (same dict as returned by dataset[i])
# and iterate over the dataset by calling the function with each example.

# Let's print the length of each `context` string in our subset of the dataset
# (10% of the validation i.e. 1057 examples)

dataset.map(lambda example: print(len(example['context']), end=','))

0it [00:00, ?it/s]

775,775,775,775,775,775,775,775,775,775,775,775,775,775,775,775,775,775,775,775,775,775,775,775,775,775,775,775,775,775,775,775,637,637,637,637,637,637,637,637,637,637,637,637,637,637,637,637,637,637,637,637,637,637,637,637,637,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,394,394,394,394,394,394,394,394,394,394,394,394,394,394,394,394,394,394,394,394,394,394,394,394,394,394,179,179,179,179,179,179,179,179,179,179,179,168,168,168,168,168,168,168,168,168,168,168,168,168,168,168,168,168,638,638,638,638,638,638,638,638,638,638,638,638,638,638,638,638,638,638,638,638,638,326,326,326,326,326,326,326,326,326,326,326,326,326,326,326,326,326,326,326,326,326,326,326,704,704,704,704,704,704,704,704,704,704,704,704,704,704,704,704,704,704,917,917,917,917,917,917,917,917,917,917,917,917,917,917,917,917,917,917,917,917,1271,1271,1271,1271,1271,1271,1271,1271,1271,1271,1271,1271,1271,1271,1271,1271,1271,1271,1271,1271,1166,1166,1166,1166,1166,116

1057it [00:00, 11478.54it/s]

,715,715,838,838,838,838,838,881,881,881,881,881,940,940,940,940,940,618,618,618,618,618,1205,1205,1205,534,534,534,534,534,757,757,757,757,757,1239,1239,1239,1239,1239,609,609,609,609,609,798,798,798,798,798,613,613,613,613,613,613,613,613,613,613,

Dataset(schema: {'title': 'string', 'context': 'string', 'question': 'string', 'answers.text': 'list<item: string>', 'answers.answer_start': 'list<item: int32>'}, num_rows: 1057)

This is basically the same as doing

```python
for example in dataset:
    function(example)
```

The above example had no effect on the dataset because the method we supplied to `.map()` didn't return a `dict` or a `abc.Mapping` that could be used to update the examples in the dataset.

In such a case, `.map()` will return the same dataset (`self`).

Now let's see how we can use a method that actually modify the dataset.

### Modifying the dataset example by example

The main interest of `.map()` is to update and modify the content of the table and leverage smart caching and fast backend.

To use `.map()` to update elements in the table you need to provide a methode with the following signature: `function(example: dict) -> dict`.

In [17]:
# Let's add a prefix 'My cute title: ' to each of our titles

def add_prefix_to_title(example):
    example['title'] = 'My cute title: ' + example['title']
    return example

dataset = dataset.map(add_prefix_to_title)

print(dataset.unique('title'))

INFO:nlp.arrow_dataset:Caching processed dataset at /Users/thomwolf/.cache/huggingface/datasets/squad/plain_text/1.0.0/cache-d99267fa77c05190a9cefa70b7a33564.arrow
1057it [00:00, 26497.84it/s]
INFO:nlp.arrow_writer:Done writing 1057 examples in 906626 bytes /Users/thomwolf/.cache/huggingface/datasets/squad/plain_text/1.0.0/cache-d99267fa77c05190a9cefa70b7a33564.arrow.


['My cute title: Super_Bowl_50', 'My cute title: Warsaw']


This call to `.map()` compute and return the updated table. It will also store the updated table in a cache file indexed by the current state and the mapped function.

A subsequent call to `.map()` (even in another python session) will reuse the cached file instead of recomputing the operation.

You can test this by running again the previous cell, you will see that the result are directly loaded from the cache and not re-computed again.

The updated dataset returned by `.map()` is (again) directly memory mapped from drive and not allocated in RAM.

The methode you provide to `.map()` should accept an input with the format of an item of the dataset: `function(dataset[0])` and return a python dict.

The columns and type of the outputs can be different than the input dict. In this case the new keys will be added as additional columns in the dataset.

The example is `updated()` with the output dictionary: `examples.update(function(example))`.

In [18]:
# Since the input example is updated with our function output,
# we can actually just return the updated 'title' field
dataset = dataset.map(lambda example: {'title': 'My cutest title: ' + example['title']})

print(dataset.unique('title'))

INFO:nlp.arrow_dataset:Caching processed dataset at /Users/thomwolf/.cache/huggingface/datasets/squad/plain_text/1.0.0/cache-b5fef4a553eb27e130e299608150b28a.arrow
1057it [00:00, 25758.39it/s]
INFO:nlp.arrow_writer:Done writing 1057 examples in 924595 bytes /Users/thomwolf/.cache/huggingface/datasets/squad/plain_text/1.0.0/cache-b5fef4a553eb27e130e299608150b28a.arrow.


['My cutest title: My cute title: Super_Bowl_50', 'My cutest title: My cute title: Warsaw']


#### Removing columns
You can also remove columns when running map with the `remove_columns=List[str]` argument.

In [19]:
# This will select the 'title' input to send to our function (as only field in the input)
# and replace it with the output of the method as a 'new_title' field
dataset = dataset.map(lambda example: {'new_title': 'Wouhahh: ' + example['title']},
                     remove_columns=['title'])

print(dataset.column_names)
print(dataset.unique('new_title'))

INFO:nlp.arrow_dataset:Caching processed dataset at /Users/thomwolf/.cache/huggingface/datasets/squad/plain_text/1.0.0/cache-ca7a1d9079c4c5bcb746ee34fb9587b6.arrow
1057it [00:00, 24044.66it/s]
INFO:nlp.arrow_writer:Done writing 1057 examples in 934108 bytes /Users/thomwolf/.cache/huggingface/datasets/squad/plain_text/1.0.0/cache-ca7a1d9079c4c5bcb746ee34fb9587b6.arrow.


['context', 'question', 'answers.text', 'answers.answer_start', 'new_title']
['Wouhahh: My cutest title: My cute title: Super_Bowl_50', 'Wouhahh: My cutest title: My cute title: Warsaw']


#### Using examples indices
With `with_indices=True`, dataset indices (from `0` to `len(dataset)`) will be supplied to the function which must thus have the following signature: `function(example: dict, indice: int) -> dict`

In [20]:
# This will add the index in the dataset to the 'question' field
dataset = dataset.map(lambda example, idx: {'question': f'{idx}: ' + example['question']},
                      with_indices=True)

print('\n'.join(dataset['question'][:5]))

INFO:nlp.arrow_dataset:Caching processed dataset at /Users/thomwolf/.cache/huggingface/datasets/squad/plain_text/1.0.0/cache-0be0bac8de2ddc42bcc650f62286e93a.arrow
1057it [00:00, 24633.58it/s]
INFO:nlp.arrow_writer:Done writing 1057 examples in 939340 bytes /Users/thomwolf/.cache/huggingface/datasets/squad/plain_text/1.0.0/cache-0be0bac8de2ddc42bcc650f62286e93a.arrow.


0: Which NFL team represented the AFC at Super Bowl 50?
1: Which NFL team represented the NFC at Super Bowl 50?
2: Where did Super Bowl 50 take place?
3: Which NFL team won Super Bowl 50?
4: What color was used to emphasize the 50th anniversary of the Super Bowl?


### Modifying the dataset with batched updates

`.map()` can also work with batch of examples (slices of the dataset).

This is particularly interesting if you have a function that can handle batch of inputs like the tokenizers of HuggingFace `tokenizers`.

To work on batched inputs set `batched=True` when calling `.map()` and supply a function with the following signature: `function(examples: Dict[List]) -> Dict[List]` or, if you use indices, `function(examples: Dict[List], indices: List[int]) -> Dict[List]`).

Your function should accept an input with the format of a slice of the dataset: e.g. `function(dataset[:10])`.

In [21]:
# Let's import a fast tokenizer that can work on batched inputs
# (the 'Fast' tokenizers in HuggingFace)
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

INFO:transformers.file_utils:PyTorch version 1.4.0 available.
INFO:transformers.file_utils:TensorFlow version 2.1.0 available.
INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at /Users/thomwolf/.cache/torch/transformers/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1


In [22]:
# Now let's batch tokenize our dataset 'context'
dataset = dataset.map(lambda example: tokenizer.batch_encode_plus(example['context']),
                      batched=True)

print("dataset[0]", dataset[0])

INFO:nlp.arrow_dataset:Caching processed dataset at /Users/thomwolf/.cache/huggingface/datasets/squad/plain_text/1.0.0/cache-522d9b9a4a2809ce238350c7bff6e151.arrow
100%|██████████| 2/2 [00:00<00:00, 22.66it/s]
INFO:nlp.arrow_writer:Done writing 1057 examples in 4810488 bytes /Users/thomwolf/.cache/huggingface/datasets/squad/plain_text/1.0.0/cache-522d9b9a4a2809ce238350c7bff6e151.arrow.


dataset[0] {'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.', 'question': '0: Which NFL team represented the AFC at Super Bowl 50?', 'answers.text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'], 'answers.answer_start': [177, 177, 177], 'new_title': 

In [23]:
# we have added additional columns
# we could have replaced the dataset with `remove_columns=True`
print(dataset.column_names)

['context', 'question', 'answers.text', 'answers.answer_start', 'new_title', 'input_ids', 'token_type_ids', 'attention_mask']


In [24]:
# Let show a more complex processing with the full preparation of the SQuAD dataset
# for training a model from Transformers
def convert_to_features(batch):
    # Tokenize contexts and questions (as pairs of inputs)
    # keep offset mappings for evaluation
    input_pairs = list(zip(batch['context'], batch['question']))
    encodings = tokenizer.batch_encode_plus(input_pairs,
                                            pad_to_max_length=True,
                                            return_offsets_mapping=True)

    # Compute start and end tokens for labels
    start_positions, end_positions = [], []
    for i, (text, start) in enumerate(zip(batch['answers.text'], batch['answers.answer_start'])):
        first_char = start[0]
        last_char = first_char + len(text[0]) - 1
        start_positions.append(encodings.char_to_token(i, first_char))
        end_positions.append(encodings.char_to_token(i, last_char))

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})
    return encodings

dataset = dataset.map(convert_to_features, batched=True)

INFO:nlp.arrow_dataset:Caching processed dataset at /Users/thomwolf/.cache/huggingface/datasets/squad/plain_text/1.0.0/cache-400648306eb35b5141fc5835ed050f81.arrow
100%|██████████| 2/2 [00:00<00:00,  5.58it/s]
INFO:nlp.arrow_writer:Done writing 1057 examples in 21997870 bytes /Users/thomwolf/.cache/huggingface/datasets/squad/plain_text/1.0.0/cache-400648306eb35b5141fc5835ed050f81.arrow.


In [25]:
# Now our dataset comprise the labels for the start and end position
# as well as the offsets for converting back tokens
# in span of the original string for evaluation
print("column_names", dataset.column_names)
print("start_positions", dataset[:5]['start_positions'])

column_names ['context', 'question', 'answers.text', 'answers.answer_start', 'new_title', 'input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'start_positions', 'end_positions']
start_positions [34, 45, 80, 34, 98]


# Formating outputs for numpy/torch/tensorflow

Now that we have tokenized our inputs, we probably want to use this dataset in a `torch.Dataloader` or a `tf.data.Dataset`.

To be able to do this we need to tweak two things:

- format the indexing (`__getitem__`) to return numpy/pytorch/tensorflow tensors, instead of python objects, and
- format the indexing (`__getitem__`) to return only the subset of the columns that we need for our model inputs.

  We don't want the columns `id` or `title` as input sto train our model, but we could still want to keep them in the dataset, for instance for the evaluation of the model.
    
This is handled by the `.set_format(type: Union[None, str], columns: Union[None, str, List[str]])` where:

- `type` define the return type for our dataset `__getitem__` method and is one of `[None, 'numpy', 'pandas', 'torch', 'tensorflow']` (`None` means return python objects), and
- `columns` define the columns returned by `__getitem__` and takes the name of a column in the dataset or a list of columns to return (`None` means return all columns).

In [26]:
columns_to_return = ['input_ids', 'token_type_ids', 'attention_mask',
                     'start_positions', 'end_positions']

dataset.set_format(type='torch',
                   columns=columns_to_return)

# Our dataset indexing output is now ready for being used in a pytorch dataloader
print('\n'.join([' '.join((n, str(type(t)), str(t.shape))) for n, t in dataset[:10].items()]))

INFO:nlp.arrow_dataset:Set __getitem__(key) output type to torch and filter ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'] columns  (when key is int or slice).


input_ids <class 'torch.Tensor'> torch.Size([10, 451])
token_type_ids <class 'torch.Tensor'> torch.Size([10, 451])
attention_mask <class 'torch.Tensor'> torch.Size([10, 451])
start_positions <class 'torch.Tensor'> torch.Size([10])
end_positions <class 'torch.Tensor'> torch.Size([10])


In [27]:
# Note that the columns are not removed from the dataset,
# just not returned when calling __getitem__
print(dataset.column_names)

['context', 'question', 'answers.text', 'answers.answer_start', 'new_title', 'input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'start_positions', 'end_positions']


In [28]:
# We can remove the formating with `.reset_format()`
# or, identically, a call to `.set_format()` with no arguments
dataset.reset_format()

print('\n'.join([' '.join((n, str(type(t)))) for n, t in dataset[:10].items()]))

INFO:nlp.arrow_dataset:Set __getitem__(key) output type to python objects and filter no columns  (when key is int or slice).


context <class 'list'>
question <class 'list'>
answers.text <class 'list'>
answers.answer_start <class 'list'>
new_title <class 'list'>
input_ids <class 'list'>
token_type_ids <class 'list'>
attention_mask <class 'list'>
offset_mapping <class 'list'>
start_positions <class 'list'>
end_positions <class 'list'>


In [29]:
# The current format can be checked with `.format`,
# which is a dict of the type and formating
dataset.format

{'type': 'python',
 'columns': ['context',
  'question',
  'answers.text',
  'answers.answer_start',
  'new_title',
  'input_ids',
  'token_type_ids',
  'attention_mask',
  'offset_mapping',
  'start_positions',
  'end_positions']}

# Wrapping this all up

Let's wrap this all up with the full code to load and prepare SQuAD for training a PyTorch model.

In [30]:
import nlp
import torch 
from transformers import BertTokenizerFast

# Load our training dataset and tokenizer
dataset = nlp.load_dataset('squad')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

def get_correct_alignement(context, answer):
    """ Some original examples in SQuAD have indices wrong by 1 or 2 character. We test and fix this here. """
    gold_text = answer['text'][0]
    start_idx = answer['answer_start'][0]
    end_idx = start_idx + len(gold_text)
    if context[start_idx:end_idx] == gold_text:
        return start_idx, end_idx       # When the gold label position is good
    elif context[start_idx-1:end_idx-1] == gold_text:
        return start_idx-1, end_idx-1   # When the gold label is off by one character
    elif context[start_idx-2:end_idx-2] == gold_text:
        return start_idx-2, end_idx-2   # When the gold label is off by two character
    else:
        raise ValueError()

# Tokenize our training dataset
def convert_to_features(example_batch):
    # Tokenize contexts and questions (as pairs of inputs)
    input_pairs = list(zip(example_batch['context'], example_batch['question']))
    encodings = tokenizer.batch_encode_plus(input_pairs, pad_to_max_length=True)

    # Compute start and end tokens for labels using Transformers's fast tokenizers alignement methodes.
    start_positions, end_positions = [], []
    for i, (context, answer) in enumerate(zip(example_batch['context'], example_batch['answers'])):
        start_idx, end_idx = get_correct_alignement(context, answer)
        start_positions.append(encodings.char_to_token(i, start_idx))
        end_positions.append(encodings.char_to_token(i, end_idx-1))
    encodings.update({'start_positions': start_positions,
                      'end_positions': end_positions})
    return encodings

dataset['train'] = dataset['train'].map(convert_to_features, batched=True)

# Format our outputs to train a pytorch model
columns = ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions']
dataset['train'].set_format(type='torch', columns=columns)

# Instantiate a PyTorch Dataloader around our dataset
dataloader = torch.utils.data.DataLoader(dataset['train'], batch_size=8)

INFO:nlp.load:Checking /Users/thomwolf/.cache/huggingface/datasets/ee43d2be6898ebb9c2afefda4455306911d308bcf924d21c975796832cc7c114.f373b0de1570ca81b50bb03bd371604f7979e35de2cfcf2a3b4521d0b3104d9b.py for additional imports.
INFO:filelock:Lock 5890243472 acquired on /Users/thomwolf/.cache/huggingface/datasets/ee43d2be6898ebb9c2afefda4455306911d308bcf924d21c975796832cc7c114.f373b0de1570ca81b50bb03bd371604f7979e35de2cfcf2a3b4521d0b3104d9b.py.lock
INFO:nlp.load:Found main folder for dataset https://s3.amazonaws.com/datasets.huggingface.co/nlp/squad/squad.py at /Users/thomwolf/Documents/GitHub/datasets/src/nlp/datasets/squad
INFO:nlp.load:Found specific version folder for dataset https://s3.amazonaws.com/datasets.huggingface.co/nlp/squad/squad.py at /Users/thomwolf/Documents/GitHub/datasets/src/nlp/datasets/squad/c0327553d80335e3a3283527f64d9778df7ad04ab28f38148d072782712bb670
INFO:nlp.load:Found script file from https://s3.amazonaws.com/datasets.huggingface.co/nlp/squad/squad.py to /Users/

In [31]:
# Let's load a pretrained Bert model and a simple optimizer
from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained('bert-base-cased')
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json from cache at /Users/thomwolf/.cache/torch/transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.9da767be51e1327499df13488672789394e2ca38b877837e52618a67d7002391
INFO:transformers.configuration_utils:Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 28996
}

INFO:transformers.modeling_utils:loading weights file https://cdn.huggingface.co/bert-base-cased-pytorch_model.bin from cache at /Users/thomwolf/.cache/torch/transformers/d8

In [32]:
# Now let's train our model

model.train()
for i, batch in enumerate(dataloader):
    outputs = model(**batch)
    loss = outputs[0]
    loss.backward()
    optimizer.step()
    model.zero_grad()
    print(f'Step {i} - loss: {loss:.3}')
    if i > 3:
        break

Step 0 - loss: 6.15
Step 1 - loss: 6.06
Step 2 - loss: 5.57
Step 3 - loss: 5.33
Step 4 - loss: 5.18


# Metrics

`nlp` also provides easy access and sharing of metrics.

This aspect of the library is still experimental and the API may still evolve more than the datasets API.

Like datasets, metrics are added as small scripts wrapping common metrics in a common API.

There are several reason you may want to use metrics with `nlp` and in particular:

- metrics for specific datasets like GLUE or SQuAD are provided out-of-the-box in a simple, convenient and consistant way integrated with the dataset,
- metrics in `nlp` leverage the powerful backend to provide smart features out-of-the-box like support for distributed evaluation in PyTorch

## Using metrics

Using metrics is pretty simple, they have two main methods: `.compute(predictions, references)` to directly compute the metric and `.add(predictions, references)` to only store some results if you want to do the evaluation in one go at the end.

Here is a quick gist of a standard use of metrics (the simplest usage):
```python
import nlp
bleu_metric = nlp.load_metric('bleu')

# If you only have a single iteration, you can easily compute the score like this
predictions = model(inputs)
score = bleu_metric.compute(predictions, references)

# If you have a loop, you can "add" your predictions and references at each iteration instead of having to save them yourself (the metric object store them efficiently for you)
for batch in dataloader:
    model_input, targets = batch
    predictions = model(model_inputs)
    bleu.add(predictions, targets)
score = bleu_metric.compute()  # Compute the score from all the stored predictions/references
```

Here is a quick gist of a use in a distributed torch setup (should work for any python multi-process setup actually). It's pretty much identical to the second example above:
```python
import nlp
# You need to give the total number of parallel python processes (num_process) and the id of each process (process_id)
bleu = nlp.load_metric('bleu', process_id=torch.distributed.get_rank(),b num_process=torch.distributed.get_world_size())

for batch in dataloader:
    model_input, targets = batch
    predictions = model(model_inputs)
    bleu.add(predictions, targets)
score = bleu_metric.compute()  # Compute the score on the first node by default (can be set to compute on each node as well)
```

[TO BE ADDED]